In [52]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time

In [47]:
def update_fillrow():

    template = pd.read_csv('../data/update/20221116/분봉/005930_삼성전자.csv', encoding='utf-8 sig')
    template = template.set_index(['Unnamed: 0', '시간'])
    template = template.notnull().replace(True, np.NaN)
    template.to_csv('../data/fillrow_template.csv', encoding='utf-8 sig')

    for file in tqdm(os.listdir('../data/update/20221116/분봉/')):
        stock_info = pd.read_csv('../data/update/20221116/분봉/{0}'.format(file), encoding='utf-8 sig')
        stock_info = stock_info.set_index(['Unnamed: 0', '시간'])

        stock_info = pd.merge(template, stock_info, how='left', left_index=True, right_index=True)

        stock_info = stock_info.drop(columns=stock_info.columns[0:23], axis=1)

        columns_temp = stock_info.columns

        bin_columns = []
        for column in columns_temp:
            bin_columns.append(column.split('_')[0])

        stock_info.columns = bin_columns

        stock_info = stock_info.reset_index()
        stock_info = stock_info.rename(columns={'Unnamed: 0' : '날짜'})

        stock_info[['거래량','거래대금']] = stock_info[['거래량','거래대금']].fillna(0)
        stock_info = stock_info.bfill()

        stock_info.to_csv('../data/update/20221116/fillrow/{0}'.format(file), encoding='utf-8 sig')

In [46]:
update_fillrow()

In [54]:
def file_open(file_open):
    file_ = os.listdir(f'{file_open}')
    file_.sort()
    
    return file_

dir_min_update_open = '../data/update/20221116/fillrow/'   # 분봉 업데이트 저장
dir_day_update_open = '../data/update/20221116/일봉/'   # 일봉 업데이트 저장
file_min_update = file_open(dir_min_update_open)    # 분봉 업데이트 리스트
dir_concat_save      = '../data/update/20221116/concat/'     # 분봉 일봉 합친 것 저장

for min_df in tqdm(file_min_update):
        stock_min_df = pd.read_csv(f'{dir_min_update_open}{min_df}', index_col='날짜')
        stock_day_df = pd.read_csv(f'{dir_day_update_open}day_{min_df}')
        stock_day_df = stock_day_df.rename(columns={'Unnamed: 0' : '날짜'})
        
        input_list = ['Unnamed: 0', '전일대비','상장주식수','시가총액','외국인주문한도수량'
                    ,'외국인주문가능수량','외국인현보유수량','외국인현보유비율'
                    ,'수정주가일자','수정주가비율','기관순매수량','기관누적순매수량'
                    ,'등락주선','등락비율','예탁금','주식회전율','거래성립률']                                  
        list_ = ['시간','시가','고가','저가','종가','거래량','거래대금','누적체결매도수량'
                 ,'등락주선','등락비율','예탁금','주식회전율','거래성립률']
            
        stock_min_df = stock_min_df.drop(input_list, axis=1)
        stock_day_df = stock_day_df.drop(list_, axis=1)

        concat_df = pd.merge(stock_min_df, stock_day_df, on='날짜')
        
        concat_df.to_csv(f'{dir_concat_save}concat_{min_df}', encoding='utf-8 sig', index=False)



 41%|████      | 986/2422 [55:13<1:13:56,  3.09s/it]

In [49]:
stock_min_df = pd.read_csv('../data/update/20221116/fillrow/441270_파인엠텍.csv', index_col='날짜')
stock_day_df = pd.read_csv('../data/update/20221116/일봉/day_441270_파인엠텍.csv')

stock_day_df = stock_day_df.rename(columns={'Unnamed: 0' : '날짜'})

input_list = ['Unnamed: 0', '전일대비','상장주식수','시가총액','외국인주문한도수량'
            ,'외국인주문가능수량','외국인현보유수량','외국인현보유비율'
            ,'수정주가일자','수정주가비율','기관순매수량','기관누적순매수량'
            ,'등락주선','등락비율','예탁금','주식회전율','거래성립률']                                  
list_ = ['시간','시가','고가','저가','종가','거래량','거래대금','누적체결매도수량'
            ,'등락주선','등락비율','예탁금','주식회전율','거래성립률']
    
stock_min_df = stock_min_df.drop(input_list, axis=1)
stock_day_df = stock_day_df.drop(list_, axis=1)

concat_df = pd.merge(stock_min_df, stock_day_df, on='날짜')

concat_df.to_csv('../data/update/20221116/test.csv', encoding='utf-8 sig', index=False)